## 텍스트 마이닝

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd
from kiwipiepy import Kiwi
import tqdm, joblib

### 실습 데이터셋 준비

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 저장된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 네이버 블로그 xlsx 파일을 읽습니다.
df = pd.read_excel(io = 'Naver Blog Contents.xlsx')

In [ ]:
# df의 처음 5행만 출력합니다.
df.head()

In [ ]:
# df의 정보를 확인합니다.
df.info()

### 형태소 분석

In [ ]:
# 한글 형태소 분석기를 설정합니다.
kiwi = Kiwi()

In [ ]:
# 형태소 분석 예제 문장을 sen에 할당합니다.
sen = '아버지가방에들어가신다.'

In [ ]:
# 예제 문장을 형태소로 분석합니다.
tokens = kiwi.tokenize(text = sen)
tokens

In [ ]:
# 첫 번째 형태소를 선택합니다.
tokens[0]

In [ ]:
# 첫 번째 형태소에서 단어를 선택합니다.
tokens[0].form

In [ ]:
# 첫 번째 형태소에서 품사를 선택합니다.
tokens[0].tag

In [ ]:
# 반복문을 실행하여 일반명사 또는 고유명사인 단어만 출력합니다.
for token in tokens:
    if token.tag in ['NNG', 'NNP']:
        print(token.form)

In [ ]:
# 리스트 컴프리헨션으로 반복문 실행 결과를 리스트로 반환합니다.
[token.form for token in tokens if token.tag in ['NNG', 'NNP']]

### 말뭉치(corpus) 생성

In [ ]:
# 형태소 분석할 시리즈를 text에 할당합니다.
text = df['contents']

In [ ]:
# 반복 실행할 범위를 설정합니다.
n = df.shape[0]

In [ ]:
# 말뭉치를 저장할 빈 리스트를 생성합니다.
corpus = list()

In [ ]:
# 각 문서에서 형태소에 품사를 부착하고 일반명사와 고유명사만 선택하여 말뭉치에 추가합니다.
for i in tqdm.tqdm_notebook(range(n)):
        
    # 형태소를 분석하고 품사를 부착합니다.
    tokens = kiwi.tokenize(text = text.iloc[i])
    
    # 일반명사만 선택합니다.
    parsed = [token.form for token in tokens if token.tag in ['NNG', 'NNP']]
    
    # 리스트를 문자열로 변환합니다.
    parsed = ' '.join(parsed)
        
    # 말뭉치에 추가합니다.
    corpus.append(parsed)

In [ ]:
# text의 첫 번째 원소를 출력합니다.
text.iloc[0]

In [ ]:
# 말뭉치의 첫 번째 원소를 출력합니다.
corpus[0]

### 압축 파일로 저장

In [ ]:
# 말뭉치를 압축 파일로 저장합니다.
joblib.dump(value = corpus, filename = 'Corpus.z')

### n-gram 생성

In [ ]:
# 관련 라이브러리를 호출합니다.
from nltk import ngrams

In [ ]:
# 말뭉치의 첫 번째 원소로 bi-gram을 생성합니다.
bis = ngrams(sequence = corpus[0].split(), n = 2)

In [ ]:
# 리스트 컴프리헨션을 이용하여 bi-gram을 리스트로 변환합니다.
[bi for bi in bis]

In [ ]:
# 반복문을 실행하기 전에 결과를 저장할 빈 리스트를 생성합니다.
ngram = list()

In [ ]:
# bi-gram을 생성합니다.
for i in corpus:
    bis = ngrams(sequence = i.split(), n = 2)
    ngram.extend([bi for bi in bis])

In [ ]:
# ngram을 시리즈로 변환합니다.
ngram = pd.Series(data = ngram)
ngram

In [ ]:
# ngram의 빈도수를 계산하고 상위 10개만 출력합니다.
ngram.value_counts().sort_values(ascending = False).head(n = 10)

In [ ]:
# 하나로 결합할 단어 목록을 리스트로 생성합니다.
bigram = ['스탠바이 미', '미러 링', '디즈니 플러스']

In [ ]:
# 변경 전 단어를 넣으면 말뭉치에서 공백 제거하는 함수를 생성합니다.
def bigram_replace(corpus, word):
    before = word
    after = word.replace(' ', '')
    corpus = [i.replace(before, after) for i in corpus]
    return corpus

In [ ]:
# 두 단어를 하나로 결합합니다.
for i in tqdm.tqdm_notebook(bigram):
    corpus = bigram_replace(corpus, i)

In [ ]:
# 말뭉치에서 첫 번째 원소를 출력합니다.
corpus[0]

### 문서-단어 행렬(DTM) 생성: Bag of Words

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# cv 객체를 생성합니다.
cv = CountVectorizer()

In [ ]:
# 문서에 포함된 단어 빈도수로 배열을 생성합니다.
dtm = cv.fit_transform(raw_documents = corpus).toarray()

In [ ]:
# dtm에서 열이름인 단어 목록을 생성합니다.
feat_names = cv.get_feature_names_out()

In [ ]:
# dtm을 데이터프레임으로 변환합니다.
dtm = pd.DataFrame(data = dtm, columns = feat_names)

In [ ]:
# dtm의 처음 5행을 출력합니다.
dtm.head()

In [ ]:
# 단어별로 0의 비중(백분율)을 출력합니다.
sparse = (dtm == 0).mean()
sparse

In [ ]:
# sparse가 95% 이상인 희박한(sparse) 단어 개수를 확인합니다.
(sparse >= 0.95).sum()

In [ ]:
# dtm에서 희박한 단어를 삭제합니다.
dtm = dtm.loc[:, ~(sparse >= 0.95)]

In [ ]:
# dtm의 처음 5행을 출력합니다.
dtm.head()

In [ ]:
# dtm을 압축파일로 저장합니다.
joblib.dump(value = dtm, filename = 'DTM.z')

### 고빈도 단어 확인

In [ ]:
# 단어별 빈도수를 내림차순으로 정렬하고 상위 10개만 출력합니다.
dtm.sum().sort_values(ascending = False).head(n = 10)

### 시각화 옵션 및 한글 폰트 설정

In [ ]:
# 관련 라이브러리를 호출합니다.
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 그래프 크기와 해상도를 설정합니다.
plt.rc(group = 'figure', figsize = (6, 6), dpi = 100)

In [ ]:
# 한글 폰트에 없는 마이너스 기호를 축에서 제거합니다.
plt.rc(group = 'axes', unicode_minus = False)

In [ ]:
# 컴퓨터에 설치된 폰트 목록을 리스트로 생성합니다.
fontList = fm.findSystemFonts(fontext = 'ttf')
fontList

In [ ]:
# 원하는 폰트 경로를 원소로 갖는 리스트를 생성합니다.
fontPath = [font for font in fontList if 'Gamja' in font]
fontPath

In [ ]:
# 한글 폰트명을 출력합니다.
fm.FontProperties(fname = fontPath[0]).get_name()

In [ ]:
# 한글 폰트를 설정합니다.
plt.rc(group = 'font', family = 'Gamja Flower', size = 10)

### 워드 클라우드 시각화

In [ ]:
# 관련 라이브러리를 호출합니다.
from wordcloud import WordCloud

In [ ]:
# 빈 딕셔너리를 생성합니다.
freq = dict()

In [ ]:
# 워드 클라우드 시각화용 딕셔너리를 생성합니다.
for word in tqdm.tqdm_notebook(dtm.columns):
    freq[word] = dtm[word].sum()

In [ ]:
# 딕셔너리를 출력합니다.
freq

In [ ]:
# 워드 클라우드 객체를 생성합니다.
wc = WordCloud(
    font_path = fontPath[0],
    width = 800, 
    height = 800, 
    background_color = 'black', 
    colormap = 'Dark2'
)

In [ ]:
# 워드 클라우드를 시각화합니다.
wcgf = wc.generate_from_frequencies(frequencies = freq)
plt.imshow(X = wcgf)
plt.axis('off')
plt.show()

### 연관 단어 분석

In [ ]:
# 단어 간 상관계수 행렬을 생성합니다.
corMat = dtm.corr().round(4)
corMat

In [ ]:
# 키워드와 상관계수가 높은 10개의 연관 단어를 출력합니다.
assocs = corMat['고민'].sort_values(ascending = False).head(n = 10)
assocs

In [ ]:
# 키워드 연관 단어로 막대그래프를 그립니다.
sns.barplot(x = assocs.index, y = assocs, palette = 'Blues_r')
plt.title(label = '연관 단어 목록');

### End of Document